In [ ]:
# run this code to connect to google drive
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!pip install xlsxwriter # for writing multiple sheets to an excel file

In [11]:
import pandas as pd
import re

Script for aggregating data from the summary sheets across all the months.

Each sheet should contain the summary sheet of the month named 'month', with the aggregate data as the sheet at the end labelled 'all'.

The file itself should be called 'CBC_news_sentiment_summary.xlsx'

In [7]:
# get summary from each month
jan = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_01_jan_CBC_news_sentiment.xlsx'
feb = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_02_feb_CBC_news_sentiment.xlsx'
mar = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_03_mar_CBC_news_sentiment.xlsx'
apr = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_04_apr_CBC_news_sentiment.xlsx'
may = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_05_may_CBC_news_sentiment.xlsx'
jun = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_06_jun_CBC_news_sentiment.xlsx'
jul = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_07_jul_CBC_news_sentiment.xlsx'
aug = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_08_aug_CBC_news_sentiment.xlsx'
sep = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_09_sep_CBC_news_sentiment.xlsx'
oct = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_10_oct_CBC_news_sentiment.xlsx'
nov = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_11_nov_CBC_news_sentiment.xlsx'
dec = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/2023_12_dec_CBC_news_sentiment.xlsx'

jan_df = pd.read_excel(jan, sheet_name='summary', index_col=0)
feb_df = pd.read_excel(feb, sheet_name='summary', index_col=0)
mar_df = pd.read_excel(mar, sheet_name='summary', index_col=0)
apr_df = pd.read_excel(apr, sheet_name='summary', index_col=0)
may_df = pd.read_excel(may, sheet_name='summary', index_col=0)
jun_df = pd.read_excel(jun, sheet_name='summary', index_col=0)
jul_df = pd.read_excel(jul, sheet_name='summary', index_col=0)
aug_df = pd.read_excel(aug, sheet_name='summary', index_col=0)
sep_df = pd.read_excel(sep, sheet_name='summary', index_col=0)
oct_df = pd.read_excel(aug, sheet_name='summary', index_col=0)
nov_df = pd.read_excel(nov, sheet_name='summary', index_col=0)
dec_df = pd.read_excel(dec, sheet_name='summary', index_col=0)

In [8]:
jan_df.head()

,compound_max,compound_min,compound_avg,pos_comp_count,neg_comp_count,avg_prop_diff,pos_prop_direction_count,neg_prop_direction_count,avg_pos_prop_difference,avg_neg_prop_difference
quote,0.9990,-0.9990,0.265059,2639,1421,0.029764,2694,1417,0.085514,-0.074045
non_quote,1.0000,-0.9995,0.154230,2410,1734,0.007516,2429,1735,0.057656,-0.062461
speaker,0.9854,-0.9545,0.059528,1065,516,0.000000,0,0,0.000000,0.000000
verb,0.7717,-0.9702,0.002676,413,356,0.000000,0,0,0.000000,0.000000


For the summary columns:
* compound_max - take average
* compound_min - take average
* compound_avg - take average
* pos_comp_count - take sum
* neg_comp_count - take sum
* avg_prop_diff - take average
* pos_prop_direction_count - take sum
* neg_prop_direction_count - take sum
* avg_pos_prop_difference - take average
* avg_neg_prop_difference - take average

In [9]:
# sum dataframes together
months = [jan_df, feb_df, mar_df, apr_df, may_df, jun_df, jul_df, aug_df, sep_df, oct_df, nov_df, dec_df]

sum_df = months[0]
for month in months[1:]:
  sum_df = sum_df.add(month)

# take the average of certain columns
avg_cols = ['compound_max', 'compound_min', 'compound_avg', 'avg_prop_diff', 'avg_pos_prop_difference', 'avg_neg_prop_difference']

for col in avg_cols:
  sum_df[col] = sum_df[col] / 12

In [10]:
sum_df

,compound_max,compound_min,compound_avg,pos_comp_count,neg_comp_count,avg_prop_diff,pos_prop_direction_count,neg_prop_direction_count,avg_pos_prop_difference,avg_neg_prop_difference
quote,0.999233,-0.999267,0.270103,30455,16305,0.028980,30916,16194,0.084843,-0.075117
non_quote,0.999925,-0.999642,0.160577,27964,19632,0.008476,28297,19576,0.057292,-0.061801
speaker,0.988083,-0.977183,0.061287,12368,6373,0.000000,0,0,0.000000,0.000000
verb,0.835067,-0.944033,-0.002673,4599,4470,0.000000,0,0,0.000000,0.000000


In [12]:
# write results to new excel sheet
output = '/content/drive/My Drive/evaluation_quotes/CBC_sentiment_output/sentiment_summary.xlsx'

writer = pd.ExcelWriter(output, engine='xlsxwriter')

sheets = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
i = 0

for month in months:
  month.to_excel(writer, sheet_name=sheets[i])
  i += 1

sum_df.to_excel(writer, sheet_name='aggregate_all')

writer.close()